# Lab 5: Resampling & Model Selection

The goal of this lab is to give you the tools to:

 - Perform bootstrapping
 - Use Leave One Out Cross-Validation (LOOCV)
 - Perform feature selection with:
    - Best Subset Selection
    - Forward Stepwise Selection
    - Backward Stepwise Selection

In [5]:
# Our well-trusted libraries
import numpy as np
import pandas as pd

# Sci-kit learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,LinearRegression, Lasso
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score, cross_validate

# Statsmodels
import statsmodels.api as sm

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Iterator building blocks
# combinations('ABCD', 2) --> AB AC AD BC BD CD
from itertools import combinations

# Many were concerned with warnings. The short answer is that FutureWarning (most common)
# appears when a functionality is deprecated and will be replaced. Here's how to ignore them
# even if you should find a way to resolve them in a production environment.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Throughout this lab, we will use an extremely simple dataset as this helps with the intuition. In essence, we will try to determine whether a person will default (i.e., fail to make their interest or principal payments on a debt) with their account balance, income, and a student indicator. More precisely:

 - `default` (str, binary): Whether the individual defaulted
 - `student` (str, binary): Whether the individual is a student
 - `balance` (float, continuous): The individual's account balance
 - `income` (float, continuous): The individual's annual income

In [6]:
default = pd.read_csv('default.csv',index_col=0)

display(default.head())
print(default.shape, '\n')
default.value_counts('default')

,default,student,balance,income
1,No,No,729.526495,44361.625074
2,No,Yes,817.180407,12106.134700
3,No,No,1073.549164,31767.138947
4,No,No,529.250605,35704.493935
5,No,No,785.655883,38463.495879


(10000, 4) 



default
No     9667
Yes     333
Name: count, dtype: int64

Let's turn the `default` and `student` into binary variables. We're only using best practices after all!

In [7]:
encoding_dict = {'Yes': 1,'No': 0}
for col in ['default', 'student']:
    default[col] = default[col].map(encoding_dict)

default.head()

,default,student,balance,income
1,0,0,729.526495,44361.625074
2,0,1,817.180407,12106.134700
3,0,0,1073.549164,31767.138947
4,0,0,529.250605,35704.493935
5,0,0,785.655883,38463.495879


Before we go into resampling methods, let's estimate the model with the original sample (i.e, `default`) to get a sense of the estimates of the coefficients as well as the standard errors associated with them.

In [8]:
X = default[['balance', 'income']]
X = sm.add_constant(X)

y = default['default']

display(X.head(), y.head())

results = sm.Logit(y, X).fit()
print(results.summary())

,const,balance,income
1,1.0,729.526495,44361.625074
2,1.0,817.180407,12106.134700
3,1.0,1073.549164,31767.138947
4,1.0,529.250605,35704.493935
5,1.0,785.655883,38463.495879


1    0
2    0
3    0
4    0
5    0
Name: default, dtype: int64

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Fri, 09 Feb 2024   Pseudo R-squ.:                  0.4594
Time:                        10:48:10   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
balance        0.0056      0

Here, we are interested in the estimate and the standard error of each coefficient. We will now see how resampling methods perform while keeping this as our initial finding.

## Bootstrapping

Bootstrapping is a method for estimating sampling distributions of an estimator by resampling with replacement form the original sample. It is a flexible way to quantify uncertainty associated with an estimator or a model.

We use bootstrapping to estimate hard to calculate standard errors (or confidence intervals). It is also used when the population is too small to generate a large sample.

Regarding the latter, bootstrapping is especially useful because it allows us to mimic the process of selecting many observations from the population.

Throughout, we will use the following terminology when referring to the samples:

- **Original Sample:** The one sample of size `n` we have from the population
- **Subsample:** We draw `B` subsamples (of size `n`) with replacement from the original sample

The bootstrap algorithm goes as follows:

1. Obtain the original sample from the population
2. Draw a subsample from the original sample
3. Compute the estimate ($ \hat{\alpha}_{b}^{*} $) from the subsample
4. Repeat steps 2 and 3 `B` times
5. Compute $ SE_{B}(\hat{\alpha}) = \sqrt{\frac{1}{B-1} \sum_{b=1}^{B} (\hat{\alpha}_{b}^{*} - \bar{\alpha}_{b}^{*})^{2}} $ where $ \bar{\alpha}_{b}^{*} = \frac{\sum_{b=1}^{B} \hat{\alpha}_{b}^{*}}{B}$

![Bootstrap Algorithm](bootstrap_algorithm.png)

The intuition behind the bootstrap is that we approximate the real distribution in the population by the original sample. This of course relies on the distribution in the original sample to approximate the population (which increases with n).

It is hard to overstate the importance of the bootstrap. As Efron and Hastie write, “Bootstrap confidence intervals are neither exact nor optimal, but aim instead for a wide applicability, combined with near-exact accuracy”.

There is of course the debate about how many bootstrap replications, B, we should use. Ideally, an infinite amount but that is not feasible because of computational restraints. Therefore, we will settle for “as many as possible”.


Anyways, let's get into the code to replicate the bootstrap algorithm we just looked at. Step 1 is already done as `default` is our original sample of length `n = 10000`. Hence, we move on to step 2 which is to draw a subsample with `n` observations from the original sample.

To do this, we will get randomly get the indices of `n` observations with replacement from the original sample. This means that an index can appear 0, 1, or multiple times in our subsample.

Note that we could also directly draw a subsample (i.e., make a dataframe) but we decided to do it with indices here and build the dataframe only when we need it in the `boot_fn` function.

In [9]:
def get_indices(data, n):
    '''
    Generates a random subsample (i.e., its indices)
    with replacement consisting of n observations each.

    Inputs:
        - data (pd.Dataframe): data to sample from
        - n (int): numeber of observations in the sample
    
    Output:
        - indices (np.ndarray): array of indices forming
            the samples
    '''
    assert type(data) == pd.DataFrame
    assert type(n) == int, 'n must be an integer'

    indices = np.random.choice(
        data.index,         # Indices as the input
        int(n),             # Number of indices per sample
        replace=True        # Draw samples with replacement
    )
    return indices

help(get_indices)
get_indices(default, 5)

Help on function get_indices in module __main__:

get_indices(data, n)
    Generates a random subsample (i.e., its indices)
    with replacement consisting of n observations each.
    
    Inputs:
        - data (pd.Dataframe): data to sample from
        - n (int): numeber of observations in the sample
    
    Output:
        - indices (np.ndarray): array of indices forming
            the samples



array([8701, 6455, 1622, 7076, 8228])

Remember, there is replacement, meaning that an observation can occur 0, 1, or multiple times in the subsample. Here's proof that it works:

In [10]:
np.asarray(np.unique(get_indices(default, 10000), return_counts=True)).T[-10:]

array([[9984,    2],
       [9985,    2],
       [9986,    2],
       [9989,    1],
       [9990,    1],
       [9991,    2],
       [9992,    1],
       [9993,    1],
       [9995,    1],
       [9999,    4]])

Now, we are already onto step 3, which is to estimate the coefficients with the subsample. Essentially, we do the exact same thing as previously when we used `logit` to get a sense of the ballpark in which the estimates and their standard error lie.

In [11]:
# similar to boot.fn in the exercise
def boot_fn(data, index, constant=True, features=['balance','income'], target='default'):
    '''
    Runs a logistic regression (with a constant) on only the specified
    indices within the data (i.e., on a single subsample).

    Inputs:
        - data (pd.Dataframe): data to sample from
        - indices (np.ndarray): array of indices forming the samples
        - features (lst of str): the name of the features
        - target (str): the name of the target

    Output:
        - coefficients (lst of float): the coefficients
    '''
    X = data[features].loc[index]
    if constant:
        X = sm.add_constant(X)
    y = data[target].loc[index]
    
    lr = sm.Logit(y,X).fit(disp=0)
    coefficients = [lr.params[0], lr.params[1], lr.params[2]]

    return coefficients

intercept, coef_balance, coef_income = boot_fn(default, get_indices(default, 10000))
print(f'Coefficients of a single subsample:\n\tIntercept:\t{round(intercept, 2)}\n\tBalance:\t{round(coef_balance, 2)}\n\tIncome:\t\t{round(coef_income, 2)}')

Coefficients of a single subsample:
	Intercept:	-11.05
	Balance:	0.01
	Income:		0.0


Thus, `boot_fn` returns the coefficients when running a logistic regression on the indices we give it. As seen, we will use the `get_incidces` function to generate the subsamples (of size `n = len(default)` from `default` with replacement) we will use in the `boot_fn`. It is all coming together!

Essentially, we know have the building blocks to retrieve the coefficients of a single subsample. Hence, we can now turn our attention to steps 4 and 5.

In step 4, we want to create subsamples and estimate their coefficients before getting the (mean and) standard deviation of those coefficients in step 5. We will do both in the `boot` function.

In [42]:
def boot(data, func, B):
    '''
    Executing a bootstrap over B subsamples
    to estimate the (mean of) coefficients
    and their associated standard errors.

    Inputs:
        - data (pd.Dataframe): data to sample from
        - func (fn): function executing the regression
        - B (int): number of subsamples
    
    Ouput:
        - restults (dict of dicts): bootstrapped coefficients
            and the associated standard errors
    '''
    # Step 4
    coef_intercept = []
    coef_balance = []
    coef_income = []

    coefs = ['intercept', 'balance', 'income']
    output = {coef: [] for coef in coefs}
    for i in range(B):
        reg_out = func(data, get_indices(data, len(data)))
        for i, coef in enumerate(coefs):
            output[coef].append(reg_out[i])

    # Step 5
    results = {}
    for coef in coefs:
        results[coef] = {
            'estimate': np.mean(output[coef]),
            'std_err': np.std(output[coef])
        }
    
    return results

We can finally run our own bootstrap!

In [43]:
results = boot(default, boot_fn, 1000)
for i, x in results.items():
    print(f"{i.capitalize()}:\n\tEstimate: {x['estimate']}\n\tStandard Error: {x['std_err']}")


Intercept:
	Estimate: -11.57738569123888
	Standard Error: 0.4428286034377395
Balance:
	Estimate: 0.005666259152022669
	Standard Error: 0.0002297479902383899
Income:
	Estimate: 2.090173573124014e-05
	Standard Error: 5.041863289521959e-06


We get values that are very similar to what we expected (i.e., from the logistic regression on the original sample). However, by using it, we reduce the risk of overfitting and improve the stability of our machine learning algorithms.

# Leave One Out Cross-Validation

We already went over cross-validation in previous labs. However, we have yet to cover its most extreme version; Leave One Out Cross Validation (LOOCV).

As its name suggests, LOOCV is simply cross validation where the test set contains a single observation. There are a few pros and cons as to why you would choose to use cross-validation:

- **Pros:**
    - Low Upward Bias: because you only lose one observation when fitting the model on the training data
    - Not Noisy: The validation estimate of the test error is always the same
- **Cons:**
    - Computationally Costly: Requires estimating the model `n` different times

In [12]:
squared_errors = []

# Iterate over the entire dataset one observation at a time
print('Iternation Milestones:...')
for i in default.index:
    # All except observation i is our training set
    train = default.iloc[default.index != i,:]
    # Only observation i is our test set
    test = default.iloc[default.index == i,:]

    # Fit the model and gather the squared error
    ols = LinearRegression().fit(train[['balance', 'income']], train['default'])
    test_predicted = ols.predict(test[['balance', 'income']])
    test_actual = test[['default']]
    squared_error = np.power(test_predicted - test_actual, 2)

    # Store the squared error
    squared_errors.append(squared_error.values[0][0])

    if i % 1000 == 0:
        print(i)

print(f'\n{squared_errors[:10]}\n')

# From the squared errors, get the Mean Squared Error (MSE)
print(f'MSE using LOOCV: {round(np.mean(squared_errors), 5)}')

Iternation Milestones:...
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000

[0.0005927380823778676, 0.00044313658761178763, 0.004082543662994613, 3.6509339069981414e-05, 0.0008426104370810332, 0.001051824157357238, 0.0007861782470870665, 0.0005042958841561052, 0.006093171030518951, 0.006208461720247738]

MSE using LOOCV: 0.02824


The interpretation stays the same as previously. However, notice how certain errors are way larger than others? Those are outliers, but they average out over the size of dataset so we don't need to give them much attention.

Let's make a quick step back to k-fold cross validation. An alternative way of doing it is with the `cross_val_score` that is also in the sci-kit learn library. Here is how to use it.

In [13]:
### CC UPDATE:  In this example, there is NO shuffling before the folds are determined, 
### so 5FCV returns the same thing regardless of seed

# In k-fold CV, the seed matters. But not so much if our dataset is big enough
np.random.seed(5)

X = default[['balance','income']]
Y = default['default']
ols = LinearRegression()

cv_scores = cross_val_score(
    ols,    # Specified model
    X,      # Features
    Y,      # Target
    cv = 5, # Number of folds
    scoring = 'neg_mean_squared_error'  # Metric
)

print(cv_scores)

print(np.mean(-cv_scores))

[-0.02939107 -0.02840277 -0.02662848 -0.02813761 -0.02856849]
0.02822568373730826


In [14]:
### CC UPDATE:  In this example, there IS shuffling before the folds are determined, 
### so 5FCV returns (slightly) different results depending on seed

from sklearn.model_selection import KFold

# In k-fold CV, the seed matters. But not so much if our dataset is big enough
np.random.seed(5)

X = default[['balance','income']]
Y = default['default']
ols = LinearRegression()

cv = KFold(
    5,              # Number of folds  
    shuffle=True,   # Randomizes observations into folds if true
    random_state=9  # Affects how randomization occurs
)  

cv_scores = cross_val_score(
    ols,     # Specified model
    X,       # Features
    Y,       # Target
    cv = cv, # Number of folds
    scoring = 'neg_mean_squared_error'  # Metric
)

print(cv_scores)

print(np.mean(-cv_scores))

[-0.02271776 -0.03038288 -0.02430095 -0.03140294 -0.0325379 ]
0.02826848555621416


## Best Subset Selection

We now switch gears to feature selection. First, let's find the absolute best subset of feature for our dataset.

The best subset is defined as the subset of features that predict the target the best. In other words, out of all possible combinations of features, which one has the lowest test error.

Likely, the subset of features that will perform best does not contain all available features (model complexity is too high). This is because using all features often leads to overfitting as irrelevant variables improve the training error but harm the test error. Another advantage of feature selection is that by reducing the number of features used, the interpretability of the model increases.

Anyways, first we need to get a list of all possible combinations.

In [16]:
def create_comb_betweenX(X):
    '''
    Returns a list with all possible combinations
    of a given list of features.

    Input:
        - X (lst of str): the features

    Output:
        - comb (lst of lst): All possible
            combinations
    '''
    interm = []
    for i in range(1, len(X)+1):
        interm.extend(list(combinations(X, i)))

    comb = []
    for i in interm:
        comb.append(list(i))

    return comb

comb = create_comb_betweenX(['student', 'balance', 'income'])
print(comb)

[['student'], ['balance'], ['income'], ['student', 'balance'], ['student', 'income'], ['balance', 'income'], ['student', 'balance', 'income']]


We can see that we have a list of all possible combinations that contain between 1 and `p` features. (`p = len(X.columns)`)

In [17]:
#Let's run one regression for every possible combination and store its value
comb_dict = {}

for i in comb: # For every combination
    X = default[i]
    Y = default['default']
    ols = LinearRegression()
    mse = np.mean(-1*cross_val_score(ols, X, Y, cv = 5,scoring = 'neg_mean_squared_error'))
    comb_dict[str(i)] = mse

display(comb_dict)
print(min(comb_dict, key=comb_dict.get))

{"['student']": 0.03216106432340415,
 "['balance']": 0.02825514591275812,
 "['income']": 0.03218892697818138,
 "['student', 'balance']": 0.02821718688532552,
 "['student', 'income']": 0.03216771173480891,
 "['balance', 'income']": 0.02822568373730826,
 "['student', 'balance', 'income']": 0.028223354532571528}

['student', 'balance']


We can see that the combination of features that best predict whether an individual will default is `student` and `balance`. This hints that `income` is an irrelevant variable when the others are given. This suggests that `income` is highly correlated with at least one of the other variables and thus only adds noise when predicting on the test set.

Anyways, now we know which features form the best subset. However, computing all possible combinations is not always feasible. As seen in class, the more features, the larger the data, and the more computationally intensive the model, the longer it takes to compute all possible combinations. Therefore, there is a need for other feature selections methods.

## Forward Stepwise Selection

The idea of this model is that we want to add the variable that provides the biggest improvement to our model (the one that yields the smallest error). We start with a model that has a constant as its only feature and continue until we run out of variables or see an increase in our error rate.

Its algorithm is fairly intuitive:

1. For k = 0, let $ M_{0} $ denote the null model (no Xs)
2. For k = 1, ..., p:
    - Consider all p - k + 1 models that add one predictor to $ M_{k-1} $
    - Pick the best (smallest RSS or largest $ R^{2} $) of these models and call it $ M_{k} $
3. Select the single best (CV test error, $ C_{p} $, AIC, BIC, or adjusted $ R^{2} $) model among $ M_{0}$, ..., $ M_{p}$

In [18]:
# Add constant to dataframe
default['constant'] = 1 

# Specify target
Y = default['default']

# Variables to use in forward propagation
vars_left_add = ['student', 'balance', 'income'] 

# Regression type
ols = LinearRegression()

# Starting variables (constant initially)
current_vars = ['constant']

X = default[current_vars]
benchmark_error = np.mean(-1*cross_val_score(ols, X, Y, cv = 5, scoring = 'neg_mean_squared_error'))
print(' Initial run with only one var (constant term/only bias weight):', current_vars)
print('      Benchmark error:', benchmark_error)
print('')

# Keep adding the best variables (until no improvement can be made)
for iter in range(len(vars_left_add)):
    print('\033[1m'+ 'Iteration:', iter, '\033[0m')
    error_list = []
    # Iterate over all the variables left to add
    for var in vars_left_add:
        # Modify X according to current iteration
        X = default[current_vars + [var]]
        # Perform 5-fold CV to get errors
        error = np.mean(-1*cross_val_score(ols, X, Y, cv = 5, scoring = 'neg_mean_squared_error'))
        error_list.append(error)
        print(' Running model with:', current_vars + [var])
        print('      Error:', error)

    # Chose the smallest error
    min_error = min(error_list)
    chosen_col_index = error_list.index(min_error)

    # If our current smallest error is smaller than our previous error, than we add a variable
    if min_error < benchmark_error:
        print('          *** Variable selected:', vars_left_add[chosen_col_index])
        print('          *** Min error selected:', min_error)
        print('          *** Chose the variable that generated the min error + was lower than previous error')
        print('')
        # Add the variable that produced the smallest error to current_vars
        current_vars.append(vars_left_add[chosen_col_index])
        # Delete chosen variable from vars_left_add
        del vars_left_add[chosen_col_index] 
        # Update benchmark_error
        benchmark_error = min_error
    
    # Otherwise, we stop our model
    else:
        print('          \033[4m*** No variable was selected', '\033[0m')
        print('          *** Previous error rate (', benchmark_error,') is lower than smallest error rate of this iteration (', min_error ,')')
        print('          *** Break')
        break

print('')
print('Variables chosen for our model', current_vars)

 Initial run with only one var (constant term/only bias weight): ['constant']
      Benchmark error: 0.032192381250000006

Iteration: 0 
 Running model with: ['constant', 'student']
      Error: 0.03216106432340414
 Running model with: ['constant', 'balance']
      Error: 0.02825514591275812
 Running model with: ['constant', 'income']
      Error: 0.03218892697818138
          *** Variable selected: balance
          *** Min error selected: 0.02825514591275812
          *** Chose the variable that generated the min error + was lower than previous error

Iteration: 1 
 Running model with: ['constant', 'balance', 'student']
      Error: 0.028217186885325523
 Running model with: ['constant', 'balance', 'income']
      Error: 0.028225683737308254
          *** Variable selected: student
          *** Min error selected: 0.028217186885325523
          *** Chose the variable that generated the min error + was lower than previous error

Iteration: 2 
 Running model with: ['constant', 'balance

We see that forward stepwise selection chooses the model with a constant, `balance`, and `student` as the features while excluding `income`. luckily, this coincides with the best subset selection we performed previously. However, there is no guarantee that we do find the best model from all possible combinations with forward stepwise selection.

## Forward Stepwise Selection using 'SequentialFeatureSelector'

To perform forward stepwise selection using the **SequentialFeatureSelector** in Python, you can use the 'mlxtend' library. Here's an example of how to do this:

In [30]:
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Assume X is the feature matrix and y is the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Initialize the linear regression model
lr = LinearRegression()

# Instantiate SequentialFeatureSelector
sfs = SequentialFeatureSelector(lr, k_features=3, forward=True, scoring='r2', cv=5)

# Fit the data to determine the k_features which give the most optimal model performance
sfs = sfs.fit(X_train, y_train)

# Get the selected feature names
selected_features = list(sfs.k_feature_names_)

# Transform the training and test data to have the selected features
# this is like changing X to be the subset of features that you are looking at 
X_train_sfs = sfs.transform(X_train)
X_test_sfs = sfs.transform(X_test)

# Train another LinearRegression model on the selected features
lr.fit(X_train_sfs, y_train)

# Predict the target values on the test set
y_pred = lr.predict(X_test_sfs)

# Calculate the R^2 score
r2_score = lr.score(X_test_sfs, y_test)

# Print the selected features and the R^2 score
print('Selected features:', selected_features)
print('R^2:', r2_score)

Selected features: ['constant', 'balance', 'income']
R^2: 0.1292303713733266


In [22]:
# these are all the possible arguments u could use to set your error measure
from sklearn.metrics import get_scorer_names
all_scorers = get_scorer_names()
all_scorers


This demonstrates how to use the SequentialFeatureSelector to perform forward stepwise selection by iteratively adding the most statistically significant features to the model until no further improvement is observed. The k_features parameter specifies the number of features to select, and the forward parameter is set to True to indicate forward selection.

The SequentialFeatureSelector from the mlxtend library provides a convenient way to perform feature selection using forward stepwise selection in Python.

The SequentialFeatureSelector is a part of the mlxtend.feature_selection module and is used to select a subset of features from a feature matrix based on their statistical significance.

To install the 'mlxtend' library in Anaconda Jupyter, you can use the following steps:
1. Open a command prompt.
2. Execute the following commands:

**bash**

**conda install mlxtend --channel conda-forge**

This command will install the 'mlxtend' library from the conda-forge channel.


Alternatively, you can also use the following command to install 'mlxtend' using conda:

**bash** 

**conda install mlxtend**

After the installation, you can launch a Jupyter Notebook and import the 'mlxtend' library to verify that the installation was successful.These commands utilize the conda package manager to install the 'mlxtend' library from the specified channel.

## Backward Stepwise Selection
The idea of this model is that we want to remove the variable that is not adding predictive power to our model (one that we can remove without increasing our error). We start with a full model (using all of our variables) and drop variables until we see an increase in our error rate or end up with only a constant. The idea remains the exact same as in forward stepwise selection, but we start at a different point.

The algorithm goes as follows:

1. For k = p, let $ M_{p} $ denote the fully specified model (all Xs)
2. For k = p-1, ..., 0:
    - Consider all p - k + 1 models that remove one predictor from $ M_{k+1} $
    - Pick the best (smallest RSS or largest $ R^{2} $) of these models and call it $ M_{k} $
3. Select the single best (CV test error, $ C_{p} $, AIC, BIC, or adjusted $ R^{2} $) model among $ M_{0}$, ..., $ M_{p}$

In [51]:
# Add constant to dataframe
default['constant'] = 1 

# Specify the target
Y = default['default']

# Variables to remove iteratively
vars_left_to_drop = ['student', 'balance', 'income'] 

# Regression type
ols = LinearRegression()

# Starting variables (all)
current_vars = ['constant'] + vars_left_to_drop

X = default[current_vars]
benchmark_error = np.mean(-1*cross_val_score(ols, X, Y, cv = 5, scoring = 'neg_mean_squared_error'))
print(' Initial run with all vars:', current_vars)
print('      Benchmark error:', benchmark_error)
print('')


# Keep removing the worst variables (until no improvement can be made)
for iter in range(len(vars_left_to_drop)):
    print('\033[1m'+ 'Iteration:', iter, '\033[0m')
    error_list = []
    # Iterate over all the variables left to remove
    for var in vars_left_to_drop:
        # Modify X according to current iteration
        vars_to_be_used = ['constant'] + [i for i in vars_left_to_drop if i != var]
        X = default[['constant'] + [i for i in vars_left_to_drop if i != var]]
        # Perform 5-fold CV to get errors
        error = np.mean(-1*cross_val_score(ols, X, Y, cv = 5, scoring = 'neg_mean_squared_error'))
        error_list.append(error)
        print(' Running model with:', vars_to_be_used)
        print('      Error:', error)

    # Chose the largest error
    min_error = min(error_list)
    chosen_col_index = error_list.index(min_error)

    # If our current smallest error is smaller than our previous error, then we drop the variable associated with it
    if min_error < benchmark_error:
        print('          *** Will drop:', vars_left_to_drop[chosen_col_index])
        print('          *** Min error selected:', min_error)
        print('          *** Chose the variable that generated the min error + was lower than previous error')
        print('')
        # Delete chosen variable from current_vars and vars_left_to_drop
        del current_vars[chosen_col_index + 1]
        del vars_left_to_drop[chosen_col_index]
        # Update benchmark_error
        benchmark_error = min_error
    
    # If not, we keep our model
    else:
        print('          \033[4m*** No variable was selected', '\033[0m')
        print('          *** Previous error rate (', benchmark_error,') is lower than smallest error rate of this iteration (', min_error ,')')
        print('          *** Break')
        break

print('')
print('Variables chosen for our model', current_vars)
    


 Initial run with all vars: ['constant', 'student', 'balance', 'income']
      Benchmark error: 0.028223354532571465



**Results of backward stepwise selection:** 